# A1: Data Curation
Karl Stavem  
ID:  1978397

---
> TL;DR: The goal of this assignment is to construct, analyze, and publish a dataset of monthly traffic on English Wikipedia from January 1, 2008 through August 30, 2020.
---

### Step 1: Gathering the data

In [8]:
# import libraries
import os
import requests
import json

In [ ]:
# 5 JSON files
#  One file each corresponding to the 2 query types in the legacy pagecount API (desktop, mobile)
# 3 query types in the newer pageviews API (desktop, mobile wbe, mobile app)
# Jan 2008 - August 202

##### Legacy Data

In [9]:
# set the API endpoint
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

In [ ]:
# set the parameters for the query to legacy API
params_legacy = {"project" : "en.wikipedia.org",
                 "access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2001010100",
                # for end use 1st day of month following final month of data
                 "end" : "2018100100"
                    }

In [ ]:
#


### Step 2: Processing the data

### Step 3: Analyze the data

### Step 4: Document your process thoroughly

---
### Step 5: Submit the assignment

The github link for this assignment can be found here:   <a href="https://github.com/stavem/data-512/tree/main/data-512-a1">here</a>.